# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 1: Gaining Information about Naive Bayes
-----
###### Student Name(s): Akira and Callum
###### Python version: 3.7.1 from Anaconda 
###### Submission deadline: 1pm, Fri 5 Apr 2019

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

# TrAiN sEt Is ThE tEsT sEt  
Imputed train set with mode, and for test / crossval tests we ignore missing values

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
from IPython.display import display
from collections import *
import numpy as np
from math import log

########## POSSIBLE CSVs ##########
d1 =  'anneal.csv'
h1 = 'family,product-type,steel,carbon,hardness,temper_rolling,condition,formability,strength,non-ageing,surface-finish,surface-quality,enamelability,bc,bf,bt,bw-me,bl,m,chrom,phos,cbond,marvi,exptl,ferro,corr,bbvc,lustre,jurofm,s,p,shape,oil,bore,packing,class'.split(',')

d2 =  'breast-cancer.csv'
h2 = 'age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,class'.split(',')

d3 =  'car.csv'
h3 = 'buying,maint,doors,persons,lug_boot,safety,class'.split(',')

d4 =  'cmc.csv'
h4 = 'w-education,h-education,n-child,w-relation,w-work,h-occupation,standard-of-living,media-exposure,class'.split(',')

d5 =  'hepatitis.csv'
h5 = 'sex,steroid,antivirals,fatigue,malaise,anorexia,liver-big,liver-firm,spleen-palpable,spiders,ascites,varices,histology,class'.split(',')

d6 =  'hypothyroid.csv'
h6 = 'sex,on-thyroxine,query-on-thyroxine,on_antithyroid,surgery,query-hypothyroid,query-hyperthyroid,pregnant,sick,tumor,lithium,goitre,TSH,T3,TT4,T4U,FTI,TBG,class'.split(',')

d7 =  'mushroom.csv'
h7 = 'cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class'.split(',')

d8 =  'nursery.csv'
h8 = 'parents,has_nurs,form,children,housing,finance,social,health,class'.split(',')

d9 = 'primary-tumor.csv'
h9 = 'age,sex,histologic-type,degree-of-diffe,bone,bone-marrow,lung,pleura,peritoneum,liver,brain,skin,neck,supraclavicular,axillar,mediastinum,abdominal,class'.split(',')

datasets = [d1,d2,d3,d4,d5,d6,d7,d8,d9]
dataset_headers = [h1,h2,h3,h4,h5,h6,h7,h8,h9]

In [3]:
# Dictionary that holds key 'filename' and values 'attribute names'
dictionary = {}
for i in range(9):
    dictionary[datasets[i]] = dataset_headers[i]
    
########## FUNCTION DEFINITIONS ##########
# Adds columns to the dataset
def set_column(filename):
    return dictionary[filename]

# Used for print debugging - prints all the probabilities out
def printer(posteriors, prob):
    print(posteriors)
    #########################################################################
    # Let say I wanted Pr(Sex = M | hypothyroid) and Pr(Sex = M | negative)
    # Access using dict[class label][attribute][attribute value] 
    
    for class_label in posteriors:
        print(f"\nPr({class_label}) = {prob}")
        for attribute in posteriors[class_label]:
            for val in posteriors[class_label][attribute]:
                print(f"Pr({attribute} = {val} | class = {class_label}) = {posteriors[class_label][attribute][val]}")

In [4]:
# This function should open a data file in csv, and transform it into a usable format 
def preprocess(filename):
    # Read in csv and add the column header
    df = pd.read_csv(filename, header = None, names = set_column(filename))
    
    # Drop columns that only have 1 unique value for every instance 
    non_unique = df.apply(pd.Series.nunique)
    cols_to_drop = non_unique[non_unique == 1].index
    df.drop(cols_to_drop, axis=1, inplace=True)
    
    split = df.sample(frac=0.3)
    train = df.drop(split.index)
    
    test = split.sample(frac=0.5)
    split.drop(test.index, inplace=True)
    crossval = split.copy()
    
    del df, split
    
    # Impute missing values using the most common value (mode)
    # Replace missing values denoted as '?' with np.NaN, then fill it with mode
    train.replace('?', np.NaN, inplace=True)
    mode = train.mode().iloc[0]
    train = train.fillna(mode)
    
    ########## PRINT DEBUG ##########
    print(f"TRAIN DATASET has {len(train.columns)} attributes, {len(train)} instances and {len(train['class'].unique())} unique classes.")
    print(f"TEST and CROSS-VALIDATE DATASET has {len(test)} instances each.")
    # display(mode)
    if len(cols_to_drop) > 0:
          print(f"No gain (all values in the attribute are the same) with: {','.join(cols_to_drop)}")
    ##########
    
    df = {"train": train, "test": test, "crossval": crossval}
    
          
    return df

In [5]:
def shit_preprocess(filename):
    # Read in csv and add the column header
    df = pd.read_csv(filename, header = None, names = set_column(filename))
    nunique = df.apply(pd.Series.nunique)
    cols_to_drop = nunique[nunique == 1].index
    df.drop(cols_to_drop, axis=1, inplace=True)
    
    return df

In [6]:
# This function should build a supervised NB model
def train(train_set):
    
    # Initialise dictionaries to hold the Prior and Posterior probabilities
    priors = {}
    posteriors = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
    """Accessable using posteriors[class][attribute][value]"""
    
    # For each unique class label
    for label in train_set['class'].unique():    
        # Total number of instances
        N = len(train_set)
        
        # Prior = No class label / N
        prob = len(train_set.loc[train_set['class'] == label]) / N
        priors[label] = prob
        
        # For each attribute, given a fixed class
        for attribute in train_set.columns[:-1]:
            temp = train_set.loc[train_set['class'] == label, [attribute,'class']]
            
            # Number of instances corresponding to the attribute, given a fixed class
            n = len(temp)
            
            # Count all values (zzz iterative approach)
            count = Counter(temp[attribute])
            
            # Insert into posterior dictionary
            for i in count:
                posteriors[label][attribute][i] = count[i] / n
    
    ############ PRINT DEBUG
    # print(f"Verifying that Axiom 2 has not been broken: {sum(priors.values())}")
    # printer(posteriors, prob)
    ############
    
    model = {"priors": priors, "posteriors": posteriors}
    
    return model

In [7]:
def cross_validation(filename):
    df = shit_preprocess(filename)
    df_copy = df.copy()
    
    # 60 - 20 - 20 train, test, cross-val
    partitions = []
    k_inc = k = 4
    
    for i in range(k):
        partitions.append(df.sample(frac=(1.0/k_inc)))
        k_inc -= 1
        df.drop(partitions[-1].index, axis=0, inplace=True)
    
    del df
    return_dict = defaultdict(list)
    # Run train/test for EACH partition, using partition as the test data and the rest as the training
    for i in range(k):
        # Copyright _blank_ @ https://gitgud._blank_.com/_blank_
        test = partitions[i]
        train = df_copy.iloc[df_copy.index.drop(test.index.values)]
        return_dict["train"].append(train)
        return_dict["test"].append(test)

    return return_dict

############### DEBUG ##################
cross_pairs = cross_validation(datasets[-1])

In [8]:
def cross_train(cross_dict):
    models = []
    for i in range(len(cross_dict["train"])):
        models.append(train(cross_dict["train"][i]))
    return models


In [9]:
def cross_predict(cross_models, cross_pairs):
    cross_tests = cross_pairs["test"]
    # display(cross_tests)
    result = []
    for i in range(len(cross_tests)):
        # print(f"{len(cross_models)}, {len(cross_tests)}")
        result.append(predict(cross_models[i], cross_tests[i]))
    return result

In [10]:
def cross_evaluate(cross_predicted, cross_pairs):
    cross_tests = cross_pairs["test"]
    result = []
    for i in range(len(cross_tests)):
        result.append(evaluate(cross_predicted[i], cross_tests[i]))
    return result

In [11]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(model, test_set):
    priors = model["priors"]
    posteriors = model["posteriors"]
    
    # Split into attributes, labels
    test_labels = test_set['class']
    test = test_set.drop('class', axis=1)
    cols = test_set.columns
    
    # Probabilistic Smoothing
    n = len(test_labels)
    epsilon = 1e-100
    
    # Results
    prediction = {}
    
    # Predicted labels to be returned
    # Key = test instance index, Value = predicted label
    predicted_labels = {}
    
    for i in range(n):
        instance = test.iloc[i]
        for label in priors.keys():       
            prob = log(priors[label])
            for attribute in cols:
                try:
                    # If value is non missing
                    if instance[attribute] != '?':
                        prob += log(posteriors[label][attribute][instance[attribute]])
                    # Else ignore the value
                    else:
                        pass
                except:
                    # Value is 0 in the model
                    prob += log(epsilon)
            prediction[label] = prob
        
        ########## PRINT DEBUG
        # print(f"Test Instance: {i}, Predict: {max(prediction, key=prediction.get)}, Actual: {test_labels.iloc[i]}, RESULT: {max(prediction, key=prediction.get) == test_labels.iloc[i]}")
        ##########
        
        predicted_labels[i] = max(prediction, key=prediction.get)
        
    return predicted_labels

In [12]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate(predicted_labels, test_set):
    results = list()
    
    test_labels = test_set['class']
    n = len(test_labels)
    
    for i in range(n):
        if predicted_labels[i] == test_labels.iloc[i]:
            results.append(1)
        else:
            results.append(0)
    
    return results

In [13]:
# This function should calculate the Information Gain of an attribute or a set of attribute, with respect to the class
def info_gain(train_set):
    cols = train_set.columns[:-1]
    label = train_set.columns[-1]
    entropy = {}
    
    # for attribute in cols:
        # entropy[attribute] = ENTROPY(ATTRIBUTE, LABEL)
    
    return entropy

In [14]:
# CROSS VALIDATION VERSION
for data in datasets:
    print(f"Processing {data} ...")
    
    df = preprocess(data)
    same_df = shit_preprocess(data)
    
    # entropy = info_gain(df)
    
    model = train(df["train"])
    shit_model = train(same_df)
    
    # classification holds [(predicted label, actual label), results] for the test and crossval dataset
    classificiation = defaultdict(list)
    total_accuracy, total_n = 0, 0
    
    for i in ["test","crossval"]:
        predicted_labels = predict(model, df[i])
        results = evaluate(predicted_labels, df[i])
        
        n = len(results)
        correct = sum(results)
        
        total_accuracy += correct
        total_n += n
        
        classificiation[i].append(predicted_labels)
        classificiation[i].append(results)
        
        print(f"Accuracy for {i} set: %.2f%%" % (100*correct/n))
    
    A = predict(shit_model, same_df)
    B = evaluate(A, same_df)
    
    print(f"\nOverall accuracy: %.2f%%" % (100*total_accuracy/total_n))
    print("Testing Accuracy on the Training Data: %.2f%%" % (100*sum(B)/len(B)))
    # Cross validation output
    cross_pairs = cross_validation(data)    
    cross_models = cross_train(cross_pairs)
    cross_predicted = cross_predict(cross_models, cross_pairs)
    cross_evaluated = cross_evaluate(cross_predicted, cross_pairs)
    print("Testing accuracy with cross-validation: " + " ".join(["%.2f%%" % (100*sum(array) / len(array)) for array in cross_evaluated]))
        
    print("*"*40)

Processing anneal.csv ...
TRAIN DATASET has 29 attributes, 629 instances and 5 unique classes.
TEST and CROSS-VALIDATE DATASET has 134 instances each.
No gain (all values in the attribute are the same) with: product-type,m,marvi,corr,jurofm,s,p
Accuracy for test set: 99.25%
Accuracy for crossval set: 98.52%

Overall accuracy: 98.88%
Testing Accuracy on the Training Data: 99.11%
2
Testing accuracy with cross-validation: 98.66% 98.67% 99.55% 99.11%
****************************************
Processing breast-cancer.csv ...
TRAIN DATASET has 10 attributes, 200 instances and 2 unique classes.
TEST and CROSS-VALIDATE DATASET has 43 instances each.
Accuracy for test set: 79.07%
Accuracy for crossval set: 60.47%

Overall accuracy: 69.77%
Testing Accuracy on the Training Data: 75.52%
2
Testing accuracy with cross-validation: 77.78% 74.65% 68.06% 64.79%
****************************************
Processing car.csv ...
TRAIN DATASET has 7 attributes, 1210 instances and 4 unique classes.
TEST and CRO

Questions (you may respond in a cell or cells below):

1. The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.
2. The Information Gain can be seen as a kind of correlation coefficient between a pair of attributes: when the gain is low, the attribute values are uncorrelated; when the gain is high, the attribute values are correlated. In supervised ML, we typically calculate the Infomation Gain between a single attribute and the class, but it can be calculated for any pair of attributes. Using the pair-wise IG as a proxy for attribute interdependence, in which cases are our NB assumptions violated? Describe any evidence (or indeed, lack of evidence) that this is has some effect on the effectiveness of the NB classifier.
3. Since we have gone to all of the effort of calculating Infomation Gain, we might as well use that as a criterion for building a “Decision Stump” (1-R classifier). How does the effectiveness of this classifier compare to Naive Bayes? Identify one or more cases where the effectiveness is notably different, and explain why.
4. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy. How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)
5. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the Naive Bayes classifier? Explain why, or why not.
6. Naive Bayes is said to elegantly handle missing attribute values. For the datasets with missing values, is there any evidence that the performance is different on the instances with missing values, compared to the instances where all of the values are present? Does it matter which, or how many values are missing? Would a imputation strategy have any effect on this?

Don't forget that groups of 1 student should respond to question (1), and one other question of your choosing. Groups of 2 students should respond to question (1) and question (2), and two other questions of your choosing. Your responses should be about 150-250 words each.

4. Should improve since a using the training dataset to test a supervised algo is like a cheat. the supervised algo has already seen the answers and will recognise it. Using a test / cross-validation will ensure that we are left with unseen data albeit a decrease in training instances

6. performance wise there should be no difference. there is no need to impute as a missing value could be significant in implying something. 